<h1>Golden Gate Cloning</h1>


In [ ]:
from opentrons import protocol_api
metadata = {
    "protocolName": "96 well plate golden gate assembly",
    "description": "Performs golden gate assembly in a 96 well plate for a predefined set of combinations of parts. You'll need to define your combinations in advance",
    "author": "JATD"
}

requirements = {
    "robotType": "Flex", # Ensure the robot type is Flex,
    "apiLevel": "2.23",
    }


def run(protocol: protocol_api.ProtocolContext):
    tips = protocol.load_labware("opentrons_flex_96_tiprack_50ul", "C2")
    tubes = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap", "D2")
    trash = protocol.load_trash_bin("A3")

    thermocycler = protocol.load_module("thermocycler", "A1")
    thermocycler_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")
    thermocycler.open_lid()



In [ ]:
import csv
import random
from opentrons import protocol_api

metadata = {
    "protocolName": "Golden Gate Assembly with Random Combinations",
    "description": "Randomly assembles P, U, C, T, and V37M parts into a 96-well plate using PlateLayout362025.csv",
    "author": "JATD"
}

requirements = {
    "robotType": "Flex",
    "apiLevel": "2.23",
}

def run(protocol: protocol_api.ProtocolContext):
    tips = protocol.load_labware("opentrons_flex_96_tiprack_50ul", "C2")
    source_plate = protocol.load_labware("nest_96_wellplate_100ul_pcr_full_skirt", "B1")
    trash = protocol.load_trash_bin("A3")
    thermocycler = protocol.load_module("thermocycler", "A1")
    dest_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")
    pipette = protocol.load_instrument("flex_8channel_50", "left", tip_racks=[tips])

    # Parse PlateLayout362025.csv
    part_sources = {"P": [], "U": [], "C": [], "T": [], "V": []}
    with open("/data/user_storage/PlateLayout362025.csv") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row["part_type"] in part_sources:
                part_sources[row["part_type"]].append((row["part_name"], row["well"]))

    # Only use V37M for V
    v_source = [w for n, w in part_sources["V"] if n == "V37M"]
    if not v_source:
        raise Exception("V37M not found in PlateLayout362025.csv")

    # Generate random combinations for each well
    num_wells = 96
    wells = dest_plate.wells()[:num_wells]
    combinations = []
    for _ in range(num_wells):
        p = random.choice(part_sources["P"])
        u = random.choice(part_sources["U"])
        c = random.choice(part_sources["C"])
        t = random.choice(part_sources["T"])
        v = ("V37M", v_source[0])
        combinations.append([p, u, c, t, v])

    # Transfer parts to each well
    volume_per_part = 2  # µL, adjust as needed
    for well, combo in zip(wells, combinations):
        pipette.pick_up_tip()
        for part_name, src_well in combo:
            pipette.transfer(volume_per_part, source_plate.wells_by_name()[src_well], well, new_tip="never")
        pipette.drop_tip(trash)

    # Thermocycler steps (example)
    thermocycler.close_lid()
    thermocycler.set_block_temperature(37, hold_time_minutes=60)
    thermocycler.set_block_temperature(16, hold_time_minutes=10)
    thermocycler.deactivate_block()